In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**필요한 모듈 import와 app 데이터 세트 로딩**

In [ ]:
import numpy as np
import pandas as pd
import gc
import time

%matplotlib inline

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns


pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

In [ ]:
# traing 데이터
app_train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
print('Size of application_train data: ', app_train.shape)
app_train.head()

In [ ]:
# testing 데이터
app_test = pd.read_csv('../input/home-credit-default-risk/application_test.csv')
print('Size of application_test data: ', app_test.shape)
app_test.head()

# **Application_train 에 대한 EDA**

**결측지 확인하기**

In [ ]:
def missing_values(df):
    mis_val=df.isnull().sum() # 전체 결측치 개수 확인
    
    mis_val_percent=100*df.isnull().sum()/len(df) # 결측률
    
    mis_val_table=pd.concat([mis_val, mis_val_percent],axis=1) # 결측치 개수 & 결측률 테이블 만들기
    
    mis_val_table_ren_columns=mis_val_table.rename(columns={0:'Total',1:'Percent'}) # 컬럼 이름바꾸기

    mis_val_table_ren_columns=mis_val_table_ren_columns[mis_val_table_ren_columns.iloc[:,1]!=0].sort_values('Percent',ascending=False).round(1)
    # 결측치 0인 컬럼은 제외하고 정렬

    print("입력한 데이터의 전체 컬럼 개수는 "+str(df.shape[1])+"개 이다.\n"
         "그 중에서 결측치가 있는 컬럼 개수는 "+str(mis_val_table_ren_columns.shape[0])+'개 이다.') # 요약 결과 print
    
    return mis_val_table_ren_columns

In [ ]:
mis_values = missing_values(app_train) # 결측지 확인 함수 실행
mis_values.head(10)

In [ ]:
app_train.isnull().sum() # 전체 컬럼 개수는 122개, 결측지가 있는 컬럼은 총 67개

In [ ]:
mis_values = missing_values(app_test) # 결측지 확인 함수 실행
mis_values.head(10)

In [ ]:
app_test.isnull().sum() # 전체 컬럼 개수는 121개, 결측지가 있는 컬럼은 총 64개

**Target 컬럼의 분포**

In [ ]:
app_train['TARGET'].value_counts() # value 값 count

In [ ]:
app_train['TARGET'].astype(int).plot.hist(); # 불균형 데이터라는 것을 알 수 있음

**칼럼 Types**

int64, float64: 수치형 변수 & object 타입: 범주형 변수


In [ ]:
app_train.info()

In [ ]:
app_train.dtypes.value_counts() # value 값 count

In [ ]:
app_train.select_dtypes('object').apply(pd.Series.nunique, axis = 0) # object 타입중에서 unique한 개수 확인

> name_contract_type 2개 나왔으므로 unique한 값 2개

**연속형 숫자 feature들에 대한 TARGET값이 각각 0과 1일때 Histogram 시각화**

In [ ]:
def show_hist_by_target(df, columns):
    cond_1 = (df['TARGET'] == 1) # target이 1일 때
    cond_0 = (df['TARGET'] == 0) # target이 0일 때
    
    for column in columns:
        fig, ax = plt.subplots(figsize=(12, 4), nrows=1, ncols=2, squeeze=False) # plot 생성
        sns.violinplot(x='TARGET', y=column, data=df, ax=ax[0][0])
        sns.distplot(df[cond_1][column], label='1', color='red', ax=ax[0][1]) # target 1일 때
        sns.distplot(df[cond_0][column], label='0', color='blue', ax=ax[0][1]) # target 2일 때

In [ ]:
# Feature Importances 에서 중요하게 나왔던 칼럼만 가져옴
columns = ['AMT_INCOME_TOTAL','AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_ID_PUBLISH',
           'DAYS_REGISTRATION', 'DAYS_LAST_PHONE_CHANGE', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'EXT_SOURCE_1', 
           'EXT_SOURCE_2', 'EXT_SOURCE_3', 'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK', 
           'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR']

show_hist_by_target(app_train, columns)

> * EXT_SOURCE_1, EXT_SOURCE_2, EXT_SOURCE_3 조금씩 모두 차이가 있음
> * 이 값을 3차원 다항식으로 확장

# 카테고리 피처를 TARGET의 유형에 따라 비교

연속형은 histplot으로 나타내고 카테고리형은 countplot 사용

In [ ]:
cate_col = app_train.dtypes[app_train.dtypes == 'object'].index.tolist() # object 타입 컬럼만 추출
cate_col

In [ ]:
def show_category_by_target(df, columns): # 시각화로 보여주는 함수 생성
    for column in columns: 
        chart = sns.catplot(x=column, col='TARGET', data=df, kind='count') # 칼럼별로 count
        chart.set_xticklabels(rotation=65)

show_category_by_target(app_train, cate_col) # app_train 데이터에서 object 타입일 때만 시각화

**대출 횟수 대비 연체 비율의 차이**

In [ ]:
# DAYS_BIRTH의 절대값과 TARGET변수와의 상관계수
app_train['DAYS_BIRTH']=abs(app_train['DAYS_BIRTH'])
app_train['DAYS_BIRTH'].corr(app_train['TARGET'])

고객의 나이가 많을수록 대출을 상환할 가능성이 높음

In [ ]:
plt.style.use('fivethirtyeight')

# 고객 나이에 대한 히스토그램 분포 확인
plt.hist(app_train['DAYS_BIRTH']/365, edgecolor='k',bins=25)
plt.title('Age of Client');
plt.xlabel('Age (years)');
plt.ylabel('Count');

In [ ]:
plt.figure(figsize=(10,8))

# 제때 대출을 상환하는 고객의 나이 plot (TARGET=0)
sns.kdeplot(app_train.loc[app_train['TARGET']==0,'DAYS_BIRTH']/365,label='target==0')

# 제때 대출을 상환하지못하는 고객의 나이 plot (TARGET=1)
sns.kdeplot(app_train.loc[app_train['TARGET']==1,'DAYS_BIRTH']/365,label='target==1')

plt.xlabel('Age(years)');
plt.ylabel('Density');
plt.title('Distribution of Ages');

* 남성이 여성보다 높음

# 주요 컬럼들과 TARGET의 상관도 분석

In [ ]:
T_columns = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE',
               'DAYS_EMPLOYED','DAYS_ID_PUBLISH', 'DAYS_REGISTRATION', 'DAYS_LAST_PHONE_CHANGE', 'AMT_INCOME_TOTAL', 'TARGET'] # 주요 컬럼 추출

Im_col = app_train[T_columns].corr() # 주요 컬럼만 가지고 있는 데이터 셋 생성
Im_col

In [ ]:
# 시각화
plt.figure(figsize=(20,10))
sns.heatmap(Im_col, annot=True)

# 데이터 병합

In [ ]:
# TARGET변수는 train데이터에만 있지만 필요한 변수이기때문에 다시 추가
train_labels = app_train['TARGET']


# train 데이터와 test 데이터에 둘다 있는 컬럼들의 값만 가져옴
app_train, app_test = app_train.align(app_test,join='inner',axis=1)

# TARGET변수 다시 추가
app_train['TARGET'] = train_labels

print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

# 이상치 데이터 확인 및 변경

In [ ]:
(app_train['DAYS_BIRTH'] / -365).describe() # 이상치 없음

In [ ]:
app_train['DAYS_EMPLOYED'].describe() # 이상치 있음

In [ ]:
app_train['DAYS_EMPLOYED'].plot.hist(title='Days Employment Histogram')
plt.xlabel('Days Employment'); # 이상치 고객 따로 빼서 대출 상환 비율 분석

In [ ]:
anom=app_train[app_train['DAYS_EMPLOYED']==365243] # 이상치인 것
non_anom=app_train[app_train['DAYS_EMPLOYED']!=365243] # 이상치 아닌 것 

print('The non-anomalies default on %0.2f%% of loans' %(100*non_anom['TARGET'].mean()))
print('The anomalies default on %0.2f%% of loans' % (100 * anom['TARGET'].mean()))
print('There are %d anomalous days of employment' % len(anom))

이상치로 보이는 고객들은 대출을 상환하지 못할 확률이 5.4% 낮음

In [ ]:
# 이상치(365243)인 값에 대해서 True , False로 구분
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243

# 이상치를 nan값으로 대치
app_train['DAYS_EMPLOYED'].replace({365243:np.nan},inplace=True)

app_train['DAYS_EMPLOYED'].plot.hist(title='Days Employment Histogram');
plt.xlabel('Days Employment');

In [ ]:
# test 데이터에도 train 데이터와 동일하게 작업
app_test['DAYS_EMPLOYED_ANOM'] = app_test['DAYS_EMPLOYED']==365243
app_test['DAYS_EMPLOYED'].replace({365243:np.nan}, inplace=True)

# EXT_SOURCE 분석

In [ ]:
# 필요 데이터만 따로 추출
ext_d = app_train[['TARGET','EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_BIRTH']]
ext_data = ext_d.corr()
ext_data

In [ ]:
# 시각화
plt.figure(figsize=(8,6))

sns.heatmap(ext_data, cmap=plt.cm.RdYlBu_r, vmin=-0.25, annot=True, vmax=0.6)
plt.title('Correlation Heatmap');

> *  EXT_SOURCE과 TARGET 변수는 음의 관계: EXT_SOURCE값이 증가하면 대출 상환률 높음
> * source_1과 DAYS_BIRTH 연관성 높음

In [ ]:
# 시각화
plt.figure(figsize=(10,12))
plt.subplot(3,1,1);
sns.kdeplot(app_train.loc[app_train['TARGET']==0, 'EXT_SOURCE_1'],label='target==0') # target=0
sns.kdeplot(app_train.loc[app_train['TARGET']==1, 'EXT_SOURCE_1'],label='target==1'); # target=1

In [ ]:
plt.figure(figsize=(10,12))

# source_1, 2, 3 시각화
for i, source in enumerate(['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']):
    plt.subplot(3,1,i+1)
    
    sns.kdeplot(app_train.loc[app_train['TARGET']==0,source],label='target==0')
    sns.kdeplot(app_train.loc[app_train['TARGET']==1,source],label='target==1')
    
    plt.title('Distribution of %s by Target Value' % source)
    plt.xlabel('%s' %source);
    plt.ylabel('Density');
plt.tight_layout(h_pad=2.5)

* source_3이 TARGET 값에 따라 차이가 가장 큼

# 주요 Feature들에 대한 FE1

In [ ]:
# null 값 확인
app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].isnull().sum()

In [ ]:
app_train['EXT_SOURCE_1'].value_counts(dropna=False) # value 값 count & 유일값에 NaN값 포함

In [ ]:
app_train['EXT_SOURCE_2'].value_counts(dropna=False) # value 값 count & 유일값에 NaN값 포함

In [ ]:
app_train['EXT_SOURCE_3'].value_counts(dropna=False) # value 값 count & 유일값에 NaN값 포함

**데이터 가공 전 학습과 테스트용 데이터 결합**

In [ ]:
apps = pd.concat([app_train, app_test]) # train 데이터와 test 데이터 결합
print(apps.shape)

**source 피쳐 결합하여 새로운 컬럼 생성**

In [ ]:
# EXT_SOURCE_X 피처들의 평균/최대/최소/표준편차 확인 
print('### mean ###\n', app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean())
print('### max ###\n',app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].max())
print('### min ###\n',app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].min())
print('### std ###\n',app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std())

In [ ]:
apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].head() # 데이터 셋 확인

In [ ]:
# 상관관계가 높은 것에 대해 너무 가공하면 성능이 오히려 떨어질 수 있으므로 max, min 말고  mean과 std 만 실행
apps['APPS_EXT_SOURCE_MEAN'] = apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
apps['APPS_EXT_SOURCE_STD'] = apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)

In [ ]:
apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APPS_EXT_SOURCE_STD']].head(10) # 가공 데이터 확인

In [ ]:
apps['APPS_EXT_SOURCE_STD'] = apps['APPS_EXT_SOURCE_STD'].fillna(apps['APPS_EXT_SOURCE_STD'].mean()) # 결측값 채우기

**Feature 가공**

In [ ]:
# AMT_CREDIT 으로 Feature 가공
apps['APPS_ANNUITY_CREDIT_RATIO'] = apps['AMT_ANNUITY']/apps['AMT_CREDIT']
apps['APPS_GOODS_CREDIT_RATIO'] = apps['AMT_GOODS_PRICE']/apps['AMT_CREDIT']
apps['APPS_CREDIT_GOODS_DIFF'] = apps['AMT_CREDIT'] - apps['AMT_GOODS_PRICE']

In [ ]:
# AMT_INCOME_TOTAL 으로 Feature 가공
# AMT_INCOME_TOTAL 비율로 대출 금액 관련 피처 가공
apps['APPS_ANNUITY_INCOME_RATIO'] = apps['AMT_ANNUITY']/apps['AMT_INCOME_TOTAL']
apps['APPS_CREDIT_INCOME_RATIO'] = apps['AMT_CREDIT']/apps['AMT_INCOME_TOTAL']
apps['APPS_GOODS_INCOME_RATIO'] = apps['AMT_GOODS_PRICE']/apps['AMT_INCOME_TOTAL']
# 가족수를 고려한 가처분 소득 피처 가공. 
apps['APPS_CNT_FAM_INCOME_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['CNT_FAM_MEMBERS']

In [ ]:
# DAYS_BIRTH, DAYS_EMPLOYED 으로 Feature 가공
# DAYS_BIRTH, DAYS_EMPLOYED 비율로 소득/자산 관련 Feature 가공. 
apps['APPS_EMPLOYED_BIRTH_RATIO'] = apps['DAYS_EMPLOYED']/apps['DAYS_BIRTH']
apps['APPS_INCOME_EMPLOYED_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['DAYS_EMPLOYED']
apps['APPS_INCOME_BIRTH_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['DAYS_BIRTH']
apps['APPS_CAR_BIRTH_RATIO'] = apps['OWN_CAR_AGE'] / apps['DAYS_BIRTH']
apps['APPS_CAR_EMPLOYED_RATIO'] = apps['OWN_CAR_AGE'] / apps['DAYS_EMPLOYED']

In [ ]:
# 데이터 레이블 인코딩, NULL값은 LightGBM 내부에서 처리하도록 특별한 변경하지 않음.
object_columns = apps.dtypes[apps.dtypes=='object'].index.tolist()

for column in object_columns:
    apps[column] = pd.factorize(apps[column])[0]

**학습 데이터와 테스트 데이터 다시 분리**

In [ ]:
apps_train = apps[-apps['TARGET'].isnull()]
apps_test = apps[apps['TARGET'].isnull()]
apps.shape, apps_train.shape, apps_test.shape

**LGBM 학습**

In [ ]:
from sklearn.model_selection import train_test_split

ftr_app = apps_train.drop(['SK_ID_CURR', 'TARGET'], axis=1)
target_app = app_train['TARGET']

train_x, valid_x, train_y, valid_y = train_test_split(ftr_app, target_app, test_size=0.3, random_state=2020)
train_x.shape, valid_x.shape

In [ ]:
from lightgbm import LGBMClassifier

clf = LGBMClassifier(
        n_jobs=-1,
        n_estimators=1000,
        learning_rate=0.02,
        num_leaves=32,
        subsample=0.8,
        max_depth=12,
        silent=-1,
        verbose=-1
        )

clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'auc', verbose= 100, 
        early_stopping_rounds= 100)

In [ ]:
from lightgbm import plot_importance

plot_importance(clf, figsize=(16, 32))

**학습된 Classifier를 이용하여 테스트 데이터 예측하고 결과를 Kaggle로 Submit 수행**

In [ ]:
preds = clf.predict_proba(apps_test.drop(['SK_ID_CURR', 'TARGET'], axis=1))[:, 1 ]

In [ ]:
app_test['TARGET'] = preds
app_test[['SK_ID_CURR', 'TARGET']].to_csv('apps_baseline_02.csv', index=False)

# **FE2**

**인코딩**

In [ ]:
# 필요한 모듈 import
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le_count = 0

# 컬럼별로 iterate
for col in app_train:
    if app_train[col].dtype=='object':
        # 데이터타입이 object이고 값의 종류가 두개 이하일경우 (위에 type 알아볼때 unique한 값 2개)
        if len(list(app_train[col].unique())) <=2:
            
            # train과 test에 동일하게 라벨인코딩을 하기위해 train기준으로 fit
            le.fit(app_train[col])
            
            # train-set, test-set Transform
            app_train[col]=le.transform(app_train[col])
            app_test[col]=le.transform(app_test[col])
            
            # 라벨인코딩을 한 컬럼이 몇개인지 카운트
            le_count+=1
            
print('%d columns were label encoded.' % le_count)

In [ ]:
# Label-encoding적용 안한 나머지 범주형 변수에 One-hot encoding 적용
app_train=pd.get_dummies(app_train)
app_test=pd.get_dummies(app_test)

print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

**필요한 데이터 추출**

In [ ]:
# polynomial features로 새로운 데이터 셋 생성
poly_ft_train = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH', 'TARGET']]
poly_ft_test = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]

# 결측치 처리를 위한 import
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median') # 중간값으로 대체

# target 뽑아내고 지운 뒤 null 데이터 imuter
poly_target = poly_ft_train['TARGET']
poly_ft_train = poly_ft_train.drop(columns=['TARGET'])

# 새로운 데이터의 null값 대체
poly_ft_train = imputer.fit_transform(poly_ft_train)
poly_ft_test = imputer.transform(poly_ft_test)

In [ ]:
poly_ft_train # imputer 적용 후 모습

In [ ]:
poly_ft_test # imputer 적용 후 모습

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# 3차원 다항식으로 만들기
poly_transformer=PolynomialFeatures(degree=3)

In [ ]:
# train데이터 기준으로 fit
poly_transformer.fit(poly_ft_train)

# features를 transform
poly_ft_train=poly_transformer.transform(poly_ft_train)
poly_ft_test=poly_transformer.transform(poly_ft_test)
print('Polynomial Features shape: ', poly_ft_train.shape)

In [ ]:
poly_ft_train # transform 적용 후 모습

In [ ]:
# 생긴 칼럼 무엇인지 확인
poly_transformer.get_feature_names(input_features=['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH'])

> * degree를 3으로 설정하면 1차원, 2차원, 3차원 다항식에 따른 값을 추가해줌

**새로운 feature과 target의 상관관계 분석**

In [ ]:
# features 의 데이터 프레임 생성
poly_ft_train = pd.DataFrame(poly_ft_train, 
                             columns = poly_transformer.get_feature_names(['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']))

# TARGET변수 다시 추가
poly_ft_train['TARGET']=poly_target

# TARGET변수와의 상관관계 확인 (corr 사용)
poly_corrs=poly_ft_train.corr()['TARGET'].sort_values()

# 데이터의 앞 부분과 뒷 부분 보기
print(poly_corrs.head(10))
print(poly_corrs.tail(5))

In [ ]:
# Put test features into dataframe
poly_ft_test = pd.DataFrame(poly_ft_test, 
                                  columns = poly_transformer.get_feature_names(['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH'])) 
                                                                                
# 원본 train 데이터에 새로 만든 다항변수를 merge해서 새로운 데이터셋 만들기
poly_ft_train['SK_ID_CURR'] = app_train['SK_ID_CURR']
app_train_poly = app_train.merge(poly_ft_train, on = 'SK_ID_CURR', how = 'left')

# 원본 test 데이터에 새로 만든 다항변수를 merge해서 새로운 데이터셋 만들기
poly_ft_test['SK_ID_CURR'] = app_test['SK_ID_CURR']
app_test_poly = app_test.merge(poly_ft_test, on = 'SK_ID_CURR', how = 'left')

# Align the dataframes => train데이터셋 기준으로 align 
app_train_poly, app_test_poly = app_train_poly.align(app_test_poly, join = 'inner', axis = 1)

# Print out the new shapes
print('Training data with polynomial features shape: ', app_train_poly.shape)
print('Testing data with polynomial features shape:  ', app_test_poly.shape)

In [ ]:
# 추출한 값 test 데이터에 merge
poly_ft_test = pd.DataFrame(poly_ft_test, 
                                  columns = poly_transformer.get_feature_names(['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']))
         

# 원본 train 데이터에 SK_ID_CURR 가져오고 새로 만든 다항변수를 join
poly_ft_train['SK_ID_CURR'] = app_train['SK_ID_CURR']
app_train_poly = app_train.merge(poly_ft_train, on = 'SK_ID_CURR', how = 'left')

# 원본 test 데이터에 SK_ID_CURR 가져오고 새로 만든 다항변수를 join
poly_ft_test['SK_ID_CURR'] = app_test['SK_ID_CURR']
app_test_poly = app_test.merge(poly_ft_test, on = 'SK_ID_CURR', how = 'left')

# train데이터셋 기준으로 align 
app_train_poly, app_test_poly = app_train_poly.align(app_test_poly, join = 'inner', axis = 1)

# shape 확인
print('Training data with polynomial features shape: ', app_train_poly.shape)
print('Testing data with polynomial features shape:  ', app_test_poly.shape)

In [ ]:
# 테스트 실행하기 위해 복사하여 진행
app_train_domain = app_train.copy()
app_test_domain = app_test.copy()

# train데이터에 새로운 변수 추가
app_train_domain['CREDIT_INCOME_PERCENT'] = app_train_domain['AMT_CREDIT'] / app_train_domain['AMT_INCOME_TOTAL']
app_train_domain['ANNUITY_INCOME_PERCENT'] = app_train_domain['AMT_ANNUITY'] / app_train_domain['AMT_INCOME_TOTAL']
app_train_domain['CREDIT_TERM'] = app_train_domain['AMT_ANNUITY'] / app_train_domain['AMT_CREDIT']
app_train_domain['DAYS_EMPLOYED_PERCENT'] = app_train_domain['DAYS_EMPLOYED'] / app_train_domain['DAYS_BIRTH']

In [ ]:
# test데이터에 새로운 변수 추가
app_test_domain['CREDIT_INCOME_PERCENT'] = app_test_domain['AMT_CREDIT'] / app_test_domain['AMT_INCOME_TOTAL']
app_test_domain['ANNUITY_INCOME_PERCENT'] = app_test_domain['AMT_ANNUITY'] / app_test_domain['AMT_INCOME_TOTAL']
app_test_domain['CREDIT_TERM'] = app_test_domain['AMT_ANNUITY'] / app_test_domain['AMT_CREDIT']
app_test_domain['DAYS_EMPLOYED_PERCENT'] = app_test_domain['DAYS_EMPLOYED'] / app_test_domain['DAYS_BIRTH']

In [ ]:
# domain 기반으로 그래프 생성
plt.figure(figsize=(12,20))

for i, feature in enumerate(['CREDIT_INCOME_PERCENT','ANNUITY_INCOME_PERCENT','CREDIT_TERM','DAYS_EMPLOYED_PERCENT']):
    plt.subplot(4,1,i+1)
    sns.kdeplot(app_train_domain.loc[app_train_domain['TARGET']==0,feature],label='target==0') # target=0
    sns.kdeplot(app_train_domain.loc[app_train_domain['TARGET']==1,feature],label='target==1') # target=1
    
    plt.title('Distribution of %s by Target Value' % feature)
    plt.xlabel('%s' % feature);
    plt.ylabel('Density');
    
plt.tight_layout(h_pad=2.5)

> * target에 따른 변수의 분포가 큰 차이가 없음